#### FAISS: Facebook AI Similarity Search

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
file =  os.environ["FILE_PATH"]


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader

In [ ]:
loader = TextLoader(file+"sample.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
embeddigns = OpenAIEmbeddings()

In [ ]:
db = FAISS.from_documents(docs, embeddigns)
query = "What did the president say about Ketanji Brown Jackson?"
docs = db.similarity_search(query)

In [ ]:
print(docs[0].page_content)

### Using similarity_search_with_score,带分数的相似搜索，分数越低越好，说明相似度越高。

In [ ]:
docs_and_scores = db.similarity_search_with_score(query)
docs_and_scores

### Using similarity_search_by_vector,将query变成向量来直接传入，而不是字符来进行相似搜索

In [ ]:
embedding_vector = embeddigns.embed_query(query)
docs_and_scores = db.similarity_search_by_vector(embedding_vector)
docs_and_scores

### Saving and loading: 可以保存和加载FAISS索引。这很有用，所以您不必每次使用它都重新创建它。如下的"faiss_index"会生成并保存在同目录文件夹下

In [ ]:
db.save_local("faiss_index")

In [ ]:
new_db = FAISS.load_local("faiss_index", embeddigns)

In [ ]:
docs = new_db.similarity_search(query)

In [ ]:
docs[0]

### Merging： 也可以把两个向量合成一个

In [34]:
db1 = FAISS.from_texts(["foo"], embeddigns)
db2 = FAISS.from_texts(["bar"],embeddigns)


In [35]:
db1.docstore._dict

{'14e1514a-f804-4f81-a833-f3cf87357be7': Document(page_content='foo', metadata={})}

In [36]:
db2.docstore._dict

{'85ed80f6-f5a4-4c94-a806-8b6df815c235': Document(page_content='bar', metadata={})}

In [37]:
db1.merge_from(db2)

In [38]:
db1.docstore._dict

{'14e1514a-f804-4f81-a833-f3cf87357be7': Document(page_content='foo', metadata={}),
 '85ed80f6-f5a4-4c94-a806-8b6df815c235': Document(page_content='bar', metadata={})}